In [ ]:
from scripts import support, metrics, vectorization as v, aggregation
from scripts.kuroda import kuroda
from scripts.insd import insd
from scripts.nras import nras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
%matplotlib inline

In [ ]:
# Базовая таблица РЕСУРСОВ за 2011

df = pd.read_excel("./data/baz-2011_nom2016.xlsx", "ТИцп ")
df = df.iloc[1:-11,3:-13]
df = df.fillna(0)

CP_2011_codes = df.drop(df.index[[1]], axis=0) # таблица 2011 года с кодами
CP_2011 = df.drop(df.index[[0,1]], axis=0) # таблица 2011 года без кодов

CP_2011 = np.array(df.drop(df.index[[0,1]], axis=0))
CP_2011_table = CP_2011
CP_2011 = v.tovector(CP_2011).astype(float) # векторизовали

In [ ]:
df = pd.read_excel("./data/aggregated2011.xls", "ТИцп")
df = df.iloc[3:-11,3:-13]
agg_CP_2011 = np.array(df.fillna(0))

agg_CP_2011 = v.tovector(agg_CP_2011).astype(float) # векторизовали

In [ ]:
# базовая таблица РЕСУРСОВ за 2016

df = pd.read_excel("./data/baz2016.xlsx", "ТИцп")
df = df.iloc[1:-11,3:-13]
df = df.fillna(0)

CP_2016_codes = df.drop(df.index[[1]], axis=0) # таблица 2011 года с кодами
CP_2016 = df.drop(df.index[[0,1]], axis=0) # таблица 2011 года без кодов

show_CP_2016_codes = CP_2016_codes # сохранили, чтобы была возможность в любой момент посмотреть таблицу
show_CP_2016 = CP_2016
CP_2016 = np.array(df.drop(df.index[[0,1]], axis=0))
CP_2016_table = CP_2016
CP_2016 = v.tovector(CP_2016).astype(float) # векторизовали

In [ ]:
df = pd.read_excel("./data/aggregated2016.xls", "ТИцп")
df = df.iloc[3:-11,3:-13]
agg_CP_2016 = np.array(df.fillna(0))

agg_CP_2016 = v.tovector(agg_CP_2016).astype(float) # векторизовали

In [ ]:
# В строке с кодами оставим только первые два символа - по ним и будем объединять

for i in range(98):
    a = str(CP_2011_codes.ix[1][i])
    a = a[:2]
    CP_2011_codes.ix[1][i] = a
    
CP_2011_codes.columns = CP_2011_codes.iloc[0] # сделали преобразованную строку с кодами -- заголовками
CP_2011_codes = CP_2011_codes.drop(CP_2011_codes.index[0]) # избавились от отдельной строки с кодами

CP_2011_codes = CP_2011_codes.groupby(CP_2011_codes.columns, axis=1)
right_11 = CP_2011_codes.size().array.to_numpy()

CP_2011_codes = CP_2011_codes.sum()

In [ ]:
# Создаем функцию, которая будет отрезать первые две цифры кода продукта

def splitter(x):    
    a = str(x)
    a=a[:2]
    return a

df = pd.read_excel("./data/baz-2011_nom2016.xlsx", "ТИцп ") # считали столбец с кодами
df = df.iloc[3:-3,1:2]
df = df.fillna(0)
CP_2011_codes['codes_prod'] = df

CP_2011_codes['codes_prod'] = CP_2011_codes['codes_prod'].apply(lambda x: splitter(x))  # преобразовали все коды до двух цифр

CP_2011_codes = CP_2011_codes.groupby(CP_2011_codes['codes_prod'])
left_11 = CP_2011_codes.size().array.to_numpy()

#### Проверка:
Считаем файл с готовой агрегацией (`aggregated2011.xls`) и сверим нашу агрегацию с готовой.

In [ ]:
left = aggregation.make_keym(206, 59, left_11)
right = aggregation.make_keym(98, 59, right_11)

In [ ]:
g_agg = aggregation.keys_to_g(left.astype(float), right.astype(float))

In [ ]:
((np.array(g_agg @ CP_2011)-np.array(agg_CP_2011))**2).sum()

In [ ]:
((np.array(g_agg @ CP_2016)-np.array(agg_CP_2016))**2).sum()

In [ ]:
# Занулим элементы, в которых меняется знак

CP_2011[CP_2011 * CP_2016 < 0] = 0
CP_2016[CP_2011 * CP_2016 < 0] = 0

agg_CP_2011[agg_CP_2011 * agg_CP_2016 < 0] = 0
agg_CP_2016[agg_CP_2011 * agg_CP_2016 < 0] = 0

In [ ]:
# Считаем суммы по строкам и столбцам

# Для 2011 года:
rows_sums_CP_2011 = np.sum(CP_2011_table, axis=1)
columns_sums_CP_2011 = np.sum(CP_2011_table, axis=0)
totals_CP_2011 = np.concatenate((rows_sums_CP_2011, columns_sums_CP_2011)).astype(float)[np.newaxis].T

# Для 2016 года:
rows_sums_CP_2016 = np.sum(CP_2016_table, axis=1)
columns_sums_CP_2016 = np.sum(CP_2016_table, axis=0)
totals_CP_2016 = np.concatenate((rows_sums_CP_2016, columns_sums_CP_2016)).astype(float)[np.newaxis].T

In [ ]:
agg_CP_2016_w_totals = np.vstack((agg_CP_2016, totals_CP_2016))

In [ ]:
# from scripts import support, metrics, vectorization as v, aggregation

def experiment(tables_kind, method_name, bench_matrix, origin_matrix, target_matrix, experiment_type, sparsed = True, save = True, totals = False):
    """
    Получение результата эксперимента

    Parameters
    ----------
    tables_kind: string
        степень аггрегации таблиц
    method_name: string
        метод для получения новой матрицы
    bench_matrix: np.array или sparse матрица
        бенчмарк таблица
    origin_matrix: np.array или sparse матрица
        оригинальная таблица
    target_matrix: np.array или sparse матрица
        целевая таблица
    experiment_type: string
        агрегация\дезаггрегация\проекция
    sparse: bool
        использовать sparse матрицу или np.array
    Returns
    -------
    a: float
        значение метрики
    """
    if experiment_type == 'дезаггрегация':
        g = g_agg
    elif experiment_type == 'проекция':
        ttls = v.addrows((206,98))
        g = v.addcolumns((206,98), ttls)
    else:
        raise Exception("Wrong experiment_type",experiment_type)  
    

    if sparsed:
        g = csr_matrix(g)
        
    if totals:
        g = v.addrows((206,98), g, sparsed=sparsed)
        g = v.addcolumns((206,98), g, sparsed=sparsed)
    
    # For Kuroda
    if len(method_name.split('_')) > 1:
        
        method, mtype = method_name.split('_')
        method = eval(method)
        results = np.round(method(g, eval(bench_matrix), eval(origin_matrix), mtype=int(mtype), sparsed=sparsed))
    else:
        method = eval(method_name)
        results = method(g, eval(bench_matrix), eval(origin_matrix), sparsed=sparsed)
    metrics_values = metrics.get_values(np.round(results), eval(target_matrix), save=True, shape = (206,98))
    
    # Save metrics' results
    if save:
        info = {'method':method_name, 'experiment_type':experiment_type, 'target_matrix':target_matrix, 
                'origin_matrix':origin_matrix, 'bench_matrix':bench_matrix}
        info = {**info,**metrics_values}

        res_table = pd.read_excel('results/results_rus_cp.xlsx')
        res_table = res_table.append([info])
        res_table.to_excel('results/results_rus_cp.xlsx', index=None,encoding='utf-8')

    return metrics_values 

def experiments(tables_kind, bench_matrix, origin_matrix, target_matrix, experiment_type, totals = False):
    methods_list = ['nras','insd','kuroda_3','kuroda_2','kuroda_1']
    sparsed = True
    for i, method in enumerate(methods_list):
        print('Method:', method)
        if i > 2:
            sparsed = False
        experiment(tables_kind, method, bench_matrix, origin_matrix, target_matrix, experiment_type, 
                   sparsed = sparsed, totals = totals)
            

In [ ]:
#Эксперимент: дезаггрегация таблицы 2016 из таблицы 2011
#Бенчмарк: таблица 2011
#Ограничения: таблица agg_OC_2016
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('', 'CP_2011', 'agg_CP_2016', 'CP_2016', 'дезаггрегация')

In [ ]:
#Эксперимент: проекция таблицы 2016 из таблицы 2011
#Бенчмарк: таблица 2011
#Ограничения: окаймляющие итоги
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('', 'CP_2011', 'totals_CP_2016', 'CP_2016', 'проекция')

In [ ]:
#Эксперимент: проекция + дезагрегация таблицы 2016 из таблицы 2011
#Бенчмарк: таблица 2011
#Ограничения: окаймляющие итоги + таблица agg_OC_2016
#Методы: NRAS, точность 0.01, 100 иттераций, INSD, Kuroda(1,2,3)

experiments('', 'CP_2011', 'agg_CP_2016_w_totals', 'CP_2016', 'дезаггрегация', totals=True)

-----------